# 01. Conhecendo a Requests


In [92]:
import requests as req


In [93]:
res = req.get('https://api.github.com/events')
res


<Response [403]>

## Explorando a biblioteca

Essa biblioteca é usada para fazer requisições HTTP, então tudo o que usamos em requisições Web comuns funcionam aqui!

Isso é algo importante de se lembrar pois existem alguns padrões como Methods, Status Code e padrões de requisições que se mantem e existem nessa lib.

In [94]:
res.status_code


403

In [95]:
res.url


'https://api.github.com/events'

In [99]:
res.text


<Response [403]>

In [97]:
print(f'Tipo do json response (para esse caso): {type(res.json())}')
print('Amostra de um item nessa lista:')
res.json()[0]


Tipo do json response (para esse caso): <class 'dict'>
Amostra de um item nessa lista:


KeyError: 0

## Usando outras URLs

Quando queremos acessar dados diferentes em uma API alteramos a URL que estamos utilizando na requisição. Sendo que é necessário ter uma ideia da documentação da API para saber quais são os dados mais interessantes a serem utilizados.

Outro ponto interessante é se atentar a versão da API, uma API pode ter diferentes versões e isso pode impactar no uso de seus endpoints!

In [98]:
res = req.get('https://api.github.com/versions')
res.status_code


403

In [54]:
res.json()


['2022-11-28']

Por uma questão de segurança e manutenção de código iremos usar por padrão essa versão da API (2022-11-28), só iremos alterar no futuro a nível de manutençao do projeto.

Para isso usamos os headers, que são configurações adicionais que passamos junto com a request.

# 02. Extraindo os dados

## Obtendo dados com GET

In [55]:
# especificando a versão da API
headers = {'X-GitHub-Api-Version': '2022-11-28'}


In [56]:
# definicao da api de requisicao dos repos da Amazon
api_base_url = 'https://api.github.com'
owner = 'amzn'
url = f'{api_base_url}/users/{owner}/repos'


In [57]:
response = req.get(url, headers)
response.status_code


200

In [58]:
amzn_repos = response.json()
len(amzn_repos)


30

Só conseguimos acessar 30 repos pois esse endpoint tem paginação. Para conseguirmos acessar mais repos precisamos nos autenticar. Esse é um requisito dessa API.

## Autenticação

Solicitações autenticadas têm um limite de taxa mais alto. Quando um usuário faz uma solicitação autenticada, ele fornece credenciais que comprovam sua identidade, o que permite que a API confie nele e lhe conceda acesso a recursos e funcionalidades adicionais.

Além disso, a maioria das APIs estabelece limites para o número de solicitações que um usuário pode fazer em um determinado período de tempo, conhecido como 'limite de taxa'. Quando um usuário faz solicitações autenticadas, a API geralmente permite que ele faça mais solicitações em um determinado período de tempo, devido à maior confiança e credibilidade que a autenticação fornece.

In [81]:
from os import getenv
from dotenv import load_dotenv

load_dotenv()

access_token = getenv('GITHUB_ACCESS_TOKEN')
headers['Authorization'] = f'Bearer {access_token}'


## Paginando os repositórios

In [82]:
response = req.get(url, headers)


In [83]:
api_base_url = 'https://api.github.com'
owner = 'amzn'
url = f'{api_base_url}/users/{owner}/repos'


In [91]:
repos_list = []
for page_index in range(1,8):
    try:
        url_page = f'{url}?page={page_index}'
        repos = req.get(url_page, headers).json()
        
        repos_list.append(repos)
    except:
        print('Error while getting repos')
        repos_list.append(None)


repos_list

## Transformando os dados

In [78]:
repos_list


['message',
 'documentation_url',
 'message',
 'documentation_url',
 'message',
 'documentation_url',
 'message',
 'documentation_url',
 'message',
 'documentation_url',
 'message',
 'documentation_url',
 'message',
 'documentation_url']